In [1]:
# !pip install git+https://github.com/Navu45/image-merging.git
# !pip install git+https://github.com/ChaoningZhang/MobileSAM.git

In [2]:
import os
import random

import torch
from PIL import Image
from diffusers import DDIMScheduler
from torchvision.transforms.functional import to_pil_image

from models.inpaint_image import InpaintPipeline

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = InpaintPipeline.from_pretrained(
    'Fantasy-Studio/Paint-by-Example',
    torch_dtype=torch.float16,
)
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to(device)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_xformers_memory_efficient_attention()

unet\diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

You are using a model of type clip_vision_model to instantiate a model of type clip. This is not supported for all configurations of models and can yield errors.
A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


In [5]:
from models.inpaint_image import generate_mask


def save_images(*images, path=''):
    for i, image in enumerate(images):
        image.save(path + f'image{i}.png')

def create_img(source_image, target_image) -> list:
    source_image, target_image = source_image.resize((512, 512)), target_image.resize((512, 512))

    return pipe(image=source_image,
                mask_image=generate_mask(source_image, target_image, pipe),
                example_image=target_image,
                num_inference_steps=50).images[0]

In [6]:
input_folder = './cache/in/'
output_folder = './cache/out/'

paths = os.listdir(input_folder)
random.shuffle(paths)

images = [Image.open(input_folder + path) for path in paths]
generated_images, couple = [], []

for i in range(len(images)):
    for j in range(i + 1, len(images)):
        generated_images.extend([
            create_img(images[i], images[j]),
        ])

save_images(generated_images, path=output_folder)

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [9]:
save_images(*generated_images, path=output_folder)